# Model predykcji cen aut

## Importy

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Wczytanie danych

In [ ]:
df = pd.read_csv('sales_ads_train.csv')

## Analiza danych i czyszczenie 

### Usunięcie niepotrzebnych kolumn

In [ ]:
df = df.drop('Pierwszy_wlasciciel', axis=1)
df = df.drop('Data_pierwszej_rejestracji', axis=1)
df = df.drop('Emisja_CO2', axis=1)

#### Uzasadnienie: Zbyt duże braki danych

### Uzupełnienie pustych rekordów

In [ ]:
df['Kraj_pochodzenia'] = df['Kraj_pochodzenia'].fillna('brak_danych')
df['Wersja_pojazdu'] = df['Wersja_pojazdu'].fillna('brak_danych')
df['Generacja_pojazdu'] = df['Generacja_pojazdu'].fillna('brak_danych')
df['Data_publikacji_oferty'] = df['Data_publikacji_oferty'].fillna('brak_danych')
df['Lokalizacja_oferty'] = df['Lokalizacja_oferty'].fillna('brak_danych')

### Usunięcie wierszy z pustymi wartościami

In [ ]:
df.dropna(subset=['Stan', 'Marka_pojazdu', 'Model_pojazdu','Wyposazenie'], inplace=True)

### Zastąpienie wartoąci pustych, wartościami dominującymi w zbiorze

In [ ]:
mode_cols = ['Waluta', 'Rodzaj_paliwa', 'Skrzynia_biegow', 'Naped', 'Kolor', 'Typ_nadwozia', 'Liczba_drzwi']
for col in mode_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

median_cols = ['Przebieg_km', 'Moc_KM', 'Pojemnosc_cm3', 'Rok_produkcji']
for col in median_cols:
    df[col] = df[col].fillna(df[col].median())


### Feature engineering - model lepiej zrozumie dane cechy

In [ ]:
df['Wiek_pojazdu'] = 2025 - df['Rok_produkcji']
df['Moc_na_1000cm3'] = df['Moc_KM'] / (df['Pojemnosc_cm3'] / 1000)

### Encoding dla ujednolicenia wartości nieliczbowych 

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoders = {}

for col in df.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

### Macierz korelacji cech

In [ ]:
corr = df.select_dtypes(include='number').corr()

plt.figure(figsize=(12, 8))
sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm", linewidths=0.5)
plt.title("Macierz korelacji")
plt.show()

## Modelowanie z GradientBoostingRegressor

W tej sekcji przeprowadzamy optymalizację hiperparametrów modelu `GradientBoostingRegressor` 
z wykorzystaniem siatki poszukiwań (`GridSearchCV`). 

Wybrałyśmy następujące hiperparametry do strojenia:

- **n_estimators**: Liczba drzew w modelu
- **learning_rate**: Współczynnik uczenia (tempo, w jakim model uczy się na błędach)
- **max_depth**: Maksymalna głębokość drzew

Jako metrykę do oceny przyjełyśmy `neg_mean_absolute_error` – (negatywną) średnią wartość błędu bezwzględnego.


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

X_train = df.drop(columns=['Cena'])
y_train = df['Cena']

model = GradientBoostingRegressor(random_state=42)

params = {
    'n_estimators': [100, 300],
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 5]
}

grid = GridSearchCV(model, params, cv=3, scoring='neg_mean_absolute_error', n_jobs=-1)
grid.fit(X_train, y_train)

### Najlepszy model i parametry

In [ ]:
best_model = grid.best_estimator_

print("Najlepsze parametry:", grid.best_params_)

# Dane testowe

## Wczytanie danych testowych

In [ ]:
test_df = pd.read_csv("sales_ads_test.csv")

### Usuwanie niepotrzebnych kolumn

In [ ]:
test_df.drop(['Pierwszy_wlasciciel', 'Data_pierwszej_rejestracji', 'Emisja_CO2'], axis=1, inplace=True)

### Uzupełnianie braków

In [ ]:
for col in ['Kraj_pochodzenia', 'Wersja_pojazdu', 'Generacja_pojazdu', 'Data_publikacji_oferty', 'Lokalizacja_oferty']:
    test_df[col] = test_df[col].fillna('brak_danych')

test_df.dropna(subset=['Stan', 'Marka_pojazdu', 'Model_pojazdu', 'Wyposazenie'], inplace=True)

mode_cols = ['Waluta', 'Rodzaj_paliwa', 'Skrzynia_biegow', 'Naped', 'Kolor', 'Typ_nadwozia', 'Liczba_drzwi']
for col in mode_cols:
    test_df[col] = test_df[col].fillna(test_df[col].mode()[0])

median_cols = ['Przebieg_km', 'Moc_KM', 'Pojemnosc_cm3', 'Rok_produkcji']
for col in median_cols:
    test_df[col] = test_df[col].fillna(test_df[col].median())


### Feature engineering

In [ ]:
test_df['Wiek_pojazdu'] = 2025 - test_df['Rok_produkcji']
test_df['Moc_na_1000cm3'] = test_df['Moc_KM'] / (test_df['Pojemnosc_cm3'] / 1000)

### Label encoding
##### Użyłyśmy tych samych enkoderów co dla danych treningowych

In [ ]:
for col, le in label_encoders.items():
    if col in test_df.columns:
        known = set(le.classes_)

        test_df[col] = test_df[col].where(test_df[col].isin(known), 'brak_danych')

        if 'brak_danych' not in le.classes_:
            le.classes_ = np.append(le.classes_, 'brak_danych')

        test_df[col] = le.transform(test_df[col])

# Predykcja cen dla danych testowych

In [ ]:
X_test = test_df[X_train.columns]
y_pred = best_model.predict(X_test)

## Zapis wyników do pliku

In [ ]:
output = pd.DataFrame({
    'ID': test_df['ID'],
    'Cena': y_pred
})
output['Cena'] = output['Cena'].round(2)
output.to_csv('wyniki.csv', index=False)

#### Wyświetlenie najważniejszych cech dla modelu Gradient Boosting

In [ ]:
feature_importances = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': best_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(data=feature_importances, x='Importance', y='Feature')
plt.title("Ważność cech dla najlepszego modelu")
plt.tight_layout()
plt.show()
